In [4]:
!pip install requests pandas 

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.5 MB 16.7 MB/s eta 0:00:01
   ------- -------------------------------- 2.1/11.5 MB 6.5 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.5 MB 8.6 MB/s eta 0:00:01
   ----------------------------- ---------- 8.4/11.5 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 11.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests
import pandas as pd
from io import StringIO

def get_oecd_indicator(country_code, indicator_code):
    if indicator_code == "inflation":
        indicator = "CP"
        url = f"https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_STES/.{country_code}.{indicator}.TOT.IDX2015.M?startPeriod=2024-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels"
    elif indicator_code == "interest":
        indicator = "IR3TIB01.STM"
        url = f"https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_STES/.{country_code}.{indicator}?startPeriod=2024-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels"
    else:
        return None

    response = requests.get(url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        if not df.empty:
            print("Preview of fetched data:")
            display(df.tail())  # Show the last few rows
            return float(df.iloc[-1]['Value'])
        else:
            print("No data found in response.")
    else:
        print(f"Request failed with status code {response.status_code}")
    return None

def get_inflation_rate(country="USA"):
    value = get_oecd_indicator(country, "inflation")
    return {"inflation_rate": value}

def get_interest_rate(country="USA"):
    value = get_oecd_indicator(country, "interest")
    return {"interest_rate": value}


In [6]:
# Test for USA Inflation
get_inflation_rate("USA")

Request failed with status code 404


{'inflation_rate': None}

In [7]:
# Test for Germany Interest Rate
get_interest_rate("DEU")

Request failed with status code 404


{'interest_rate': None}

In [8]:
import requests
import pandas as pd
from io import StringIO

def get_cpi_oecd(country="USA"):
    url = "https://stats.oecd.org/SDMX-JSON/data/MEI_CPI/USA.CPALTT01.ST.Q/all?startTime=2023-01"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        series = data['dataSets'][0]['series']
        obs = list(series.values())[0]['observations']
        latest_key = sorted(obs.keys())[-1]
        latest_value = obs[latest_key][0]
        return {"cpi_latest": latest_value}
    else:
        print(f"Error: {response.status_code}")
        return None

# Try it:
get_cpi_oecd()


Error: 404


In [9]:
import requests
import pandas as pd
from io import StringIO

def get_cpi_oecd_csv(country_code="USA"):
    """
    Fetch latest Consumer Price Index (CPI) for the given country using OECD CSV API.
    """
    url = (
        f"https://stats.oecd.org/sdmx-json/data/MEI_CPI/{country_code}.CPALTT01.IXOB.M/all?"
        "startTime=2024-01&dimensionAtObservation=AllDimensions&contentType=csv"
    )
    
    response = requests.get(url)
    
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df = df[df["TIME"] == df["TIME"].max()]
        latest = df["Value"].values[0]
        return {"cpi_latest": latest}
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

# Example usage
print(get_cpi_oecd_csv("USA"))


Error fetching data: 404
None


In [16]:
import os
import requests

# TEMPORARILY set the API key here for testing
os.environ["FRED_API_KEY"] = "ed700d62fa43396afa35236a0dd62654"

FRED_API_KEY = os.getenv("FRED_API_KEY")

# Helper function
def get_fred_series(series_id: str):
    url = f"https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_order": "desc",  # Most recent first
        "limit": 1
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["observations"]:
            return float(data["observations"][0]["value"])
        else:
            print("⚠️ No observations found.")
    else:
        print(f"❌ HTTP Error: {response.status_code}")
    return None

# Individual economic indicators
def get_inflation_rate(country="IND"):
    value = get_fred_series("CPIAUCSL")
    return {"inflation_rate": round(value, 2) if value else "Unavailable"}

# def get_interest_rate(country="USA"):
#     value = get_fred_series("FEDFUNDS")
#     return {"interest_rate": round(value, 2) if value else "Unavailable"}

# def get_unemployment_rate(country="USA"):
#     value = get_fred_series("UNRATE")
#     return {"unemployment_rate": round(value, 2) if value else "Unavailable"}


In [17]:
get_inflation_rate()
# get_interest_rate()
# get_unemployment_rate()


{'inflation_rate': 21.48}

In [18]:
import requests

def get_worldbank_inflation(country_code="IN"):
    url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/FP.CPI.TOTL.ZG?format=json&per_page=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) == 2 and data[1]:
            value = data[1][0]["value"]
            return {"inflation_rate": round(value, 2) if value else "Unavailable"}
    return {"inflation_rate": "Unavailable"}

print(get_worldbank_inflation("IN"))  # Output: {'inflation_rate': 5.1} or similar


{'inflation_rate': 4.95}


In [19]:
import requests

def get_interest_rate(country_code="IN", indicator="FR.INR.RINR"):
    url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/{indicator}?format=json&per_page=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) == 2 and data[1]:
            value = data[1][0]["value"]
            return {"interest_rate": round(value, 2) if value else "Unavailable"}
    return {"interest_rate": "Unavailable"}

# Real interest rate
print(get_interest_rate("IN", "FR.INR.RINR"))

# Lending rate
print(get_interest_rate("IN", "FR.INR.LEND"))


{'interest_rate': 'Unavailable'}
{'interest_rate': 'Unavailable'}
